In [22]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive/Colab Notebooks/text3"
os.chdir(path)

file_pattern = "DS_Name.xlsx"
file_text = "SPBank_AccurateName.xlsx"
remove = "[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+;"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import pandas as pd
import numpy as np
import re

pattern = pd.read_excel(file_pattern)
text = pd.read_excel(file_text)

In [24]:
import difflib
 
def similarity3(s1, s2):
    s1 = re.sub(remove, " ", s1)
    s2 = re.sub(remove, " ", s2)
    return difflib.SequenceMatcher(None, s1, s2).ratio()

similarity1("Harris Bankcorp, Inc.", "ABN AMRO Asia")

0.36363636363636365

In [25]:
import difflib
 
def similarity1(s1, s2):
    s1 = re.sub(remove, " ", s1)
    s2 = re.sub(remove, " ", s2)
    return difflib.SequenceMatcher(None, s1, s2).quick_ratio()

similarity1("Harris Bankcorp, Inc.", "ABN AMRO Asia")

0.36363636363636365

In [26]:
# calculate the similarity of two company names - version 4
from gensim import corpora, models, similarities
import re

n = len(text.axes[0])
m = len(pattern.axes[0])

wordList = []
wordListCrsp = []

for row in range(n):
    txt = text['AccurateName'][row]
    txt = re.sub(remove, " ", txt)
    segList = txt.lower().split()
    wordList.append(segList) 

for row in range(m):
    pat = pattern['lender'][row]
    pat = re.sub(remove, " ", pat)
    segList = pat.lower().split()
    wordListCrsp.append(segList)
    wordList.append(segList)

# 制作字典
dictionary = corpora.Dictionary(wordList)
# print(dictionary.token2id)


# word to vec
corpus = [dictionary.doc2bow(doc) for doc in wordList]
corpusCrsp = [dictionary.doc2bow(doc) for doc in wordListCrsp]


# 获取语料库每个文档中每个词的tfidf值，即用tfidf模型训练语料库
tfidf = models.TfidfModel(corpus)

#对稀疏向量建立索引
index = similarities.SparseMatrixSimilarity(tfidf[corpusCrsp], num_features=len(dictionary.keys()))

def similarity2(txt):
    # 对测试文本分词
    txt = re.sub(remove, " ", txt)
    dic_text_list = txt.lower().split()
    
    # 制作测试文本的词袋
    doc_text_vec = dictionary.doc2bow(dic_text_list)

    # 计算similarity
    sim = index[tfidf[doc_text_vec]]    # 相当于sim = index.get_similarities(tfidf[doc_text_vec])

    return sim

In [27]:
# check sequence

def if_sequence(txt, pat):
    txt = re.sub(remove, " ", txt)
    txt_list = txt.lower().split()
    txt_list_ = list(set(txt_list))
    txt_list_.sort(key=txt_list.index)
    d = dict(zip(txt_list_, range(len(txt_list))))

    pat = re.sub(remove, " ", pat)
    pat_list = pat.lower().split()
    i = 0
    for p in pat_list:
        if p in d and d[p] >= i:
            i = d[p]
        elif p in d and d[p] < i:
            return True

    return False

In [30]:
import re

n = len(text.axes[0])
m = len(pattern.axes[0])
count = 0
output = []

# match p to all companies in text1
for row in range(n):
    nid = text['AccurateName_id'][row]
    txt = text['AccurateName'][row]

    sim2 = similarity2(txt)

    for row_ in range(m):
        lcoid = pattern['lcoid'][row_]
        pat = pattern['lender'][row_]

        sim1 = similarity1(txt, pat)
        sim3 = similarity3(txt, pat)

        output_ = nid, txt, lcoid, pat, sim1, sim2[row_], sim3, int(if_sequence(txt, pat))
        output.append(output_)

        # if sim1 >= 0.95:
        #     count += 1
        #     output_ = nid, txt, lcoid, pat, sim1, sim2[row_], int(if_sequence(txt, pat))
        #     output.append(output_)
        # elif sim1 >= 0.8 and sim2[row_] > 0.5:
        #     count += 1
        #     output_ = nid, txt, lcoid, pat, sim1, sim2[row_], int(if_sequence(txt, pat))
        #     output.append(output_)
    # print(count)

df = pd.DataFrame(output, columns=['AccurateName_id', 'AccurateName', 'lcoid', 'lender', 'sim1=quick_ratio', 'sim2=tfidf', 'sim3=ratio', 'dummy'])

In [32]:
df[1000000:].to_excel('output2.xlsx', index = None, header=True)